# Purpose

The purpose of this note book is to examine different baseline removals and peak finding methods.

# Read in data

In [2]:
#imports
import h5py
import numpy as np
import pandas as pd

In [6]:
#
filepath = r"hdf5_data\210601_NBS295-106\20210601_152616_mass-001.hdf5"

chan_dict = []
with h5py.File(filepath, 'r') as hdf_file:
    print(list(hdf_file['chan1'].keys()))
    for i, channel in enumerate(list(hdf_file.keys())):
        chan_num = channel
        chan_data = np.array(hdf_file[channel]['filt_value'])

        chan_dict.append({'channel' : chan_num, 'data' : chan_data})


channels_df = pd.DataFrame(chan_dict)
channels_df = channels_df.set_index('channel')
channels_df

['average_pulse', 'calibration', 'cuts', 'energy', 'filt_phase', 'filt_phase_corr', 'filt_value', 'filt_value_dc', 'filt_value_matched', 'filt_value_phc', 'filt_value_tdc', 'filters', 'matchX', 'matchY', 'match_goodMatches', 'match_peaks', 'min_value', 'noise_autocorr', 'noise_psd', 'peak_index', 'peak_value', 'postpeak_deriv', 'pretrig_deriv', 'pretrig_mean', 'pretrig_offset', 'pretrig_rms', 'promptness', 'ptdc_anchors', 'ptdc_baselines', 'pulse_average', 'pulse_rms', 'rise_time', 'rowcount', 'shift1', 'tdc_anchors', 'tdc_baselines', 'timestamp']


,data
channel,
chan1,"[1917.8376, 2934.2834, 1235.1354, 8561.532, 22..."
chan101,"[9945.653, 11679.518, 8148.7476, 8897.93, 1198..."
chan103,"[11883.796, 10292.644, 4876.378, 7284.5176, 14..."
chan105,"[16857.314, 13127.779, 7157.272, 8648.918, 109..."
chan107,"[8264.235, 12281.665, 9715.321, 4386.601, 9545..."
...,...
chan91,"[8327.866, 10658.192, 9713.913, 11110.397, 111..."
chan93,"[9339.398, 5042.0146, 13543.274, 771.2593, 898..."
chan95,"[1961.5458, 10050.026, 5540.5645, 8491.4, 1023..."


# Baseline Removal

## Low-binned spline

In [ ]:
# Get data from channel
x_01 = channels_df.loc['chan1']['data']
x_01 = x_01[(x_01 >=0) & (x_01 < 24000)] #Negative energy cannot be measured. There is no meaninful data above 24000 in any file.

In [ ]:
hist_01, bin_edges_01 = np.histogram(x_01, bins = 5000)

bin_edges_01 = bin_edges_01[:-1]  
df01 = pd.DataFrame({
    'Bin_Left_Edge': bin_edges_01,
    'Count': hist_01})

df01

df01.plot(drawstyle = 'steps-pre', x = 'Bin_Left_Edge', y =  'Count', logy = True, xlim = (0, 18000))

## Savgol_filter

# Peak finding